In [1]:
import os
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
from PIL import Image
import torchvision.transforms.functional as tf
from torch.utils.tensorboard import SummaryWriter
import datetime

In [2]:
torch.manual_seed(2)

In [3]:
# Number of Historical Days used as Input!
num_days_history = 84

In [4]:
base_folder = "/scratch/w28/ad0033/FD/" 

In [5]:
curr_dir = "flash_drought_ml_retrain/"
curr_folder = base_folder + curr_dir

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Device: cuda


In [7]:
# Building mask to remove invalid areas
file_path = base_folder + "flash_drought_ml/data/FD/" + os.listdir(base_folder + "flash_drought_ml/data/FD/")[0]
im = Image.open(file_path)
mask = tf.to_tensor(im)
mask[mask >= 0] = True
mask[mask < 0] = False
mask = mask.bool().to(device)
print(f"Number of valid points:, {torch.sum(mask):,}")

Number of valid points:, 152,661


In [8]:
image_shape = mask.shape

In [9]:
class CustomDataset(Dataset):
    def __init__(self, start_pt, num_pts):
        self.start_pt = start_pt
        self.num_pts = num_pts
        self.factor = 2

        base = base_folder + "flash_drought_ml/data/"
        self.input_folder = base + "input/"
        self.output_folder = base + "output/"
        self.input_files = sorted(os.listdir(self.input_folder))
        self.output_files = sorted(os.listdir(self.output_folder))
        
        months = torch.arange(12)
        months_sin = torch.sin(2 * torch.pi * months / 12).unsqueeze(1).unsqueeze(2)
        months_cos = torch.cos(2 * torch.pi * months / 12).unsqueeze(1).unsqueeze(2)
        self.months_sin = months_sin.repeat(1, image_shape[1], image_shape[2])
        self.months_cos = months_cos.repeat(1, image_shape[1], image_shape[2])
        
        y_axis = torch.arange(image_shape[1]).unsqueeze(0).unsqueeze(2).repeat(1, 1, image_shape[2]) / image_shape[1]
        x_axis = torch.arange(image_shape[2]).unsqueeze(0).unsqueeze(1).repeat(1, image_shape[1], 1) / image_shape[2]
        self.position = torch.cat([y_axis, x_axis], 0)

        end_pt = start_pt + num_pts * self.factor

        assert end_pt <= len(self.output_files), f"{end_pt} > {len(self.output_files)}"

    def __len__(self):
        return self.num_pts

    def __getitem__(self, idx):
        idx = idx * self.factor
        output_file_name = self.output_files[self.start_pt + idx]
        month = int(output_file_name[4:6])
        input = []
        for i in range(num_days_history):
            # assert self.start_pt + idx + 67 - i >= 0
            input.append(torch.load(self.input_folder + self.input_files[self.start_pt + idx + 67 - i]))
        input.append(self.months_sin[month-1].unsqueeze(0))
        input.append(self.months_cos[month-1].unsqueeze(0))
        input.append(self.position)
        input = torch.cat(input, 0)
        output = torch.load(self.output_folder + output_file_name)
        return input, output, output_file_name

In [10]:
dataloader = DataLoader(CustomDataset(22, 913), batch_size=2, shuffle=False)
train_features, train_labels, file_names = next(iter(dataloader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")
print(file_names)

Feature batch shape: torch.Size([2, 508, 676, 407])
Labels batch shape: torch.Size([2, 1, 676, 407])
('20010401.pt', '20010403.pt')


Model

In [11]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        convs = []
        bns = []
        dropouts = []
        conv_chs = [6 * num_days_history + 4, 512, 256, 128, 64, 32, 16, 8]
        for i in range(len(conv_chs) - 1):
            convs.append(nn.Conv2d(conv_chs[i], conv_chs[i + 1], kernel_size=(3,3), padding="same"))
            bns.append(nn.BatchNorm2d(conv_chs[i + 1]))
            dropouts.append(nn.Dropout2d(0.1))
        self.convs = nn.ModuleList(convs)
        self.bns = nn.ModuleList(bns)
        self.dropouts = nn.ModuleList(dropouts)
        self.act = nn.ReLU()
        self.final_conv = nn.Conv2d(conv_chs[-1], 1, kernel_size=(3,3), padding="same")
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        for conv, bn, dropout in zip(self.convs, self.bns, self.dropouts):
            x = self.act(bn(conv(x)))
            x = dropout(x)
        x = self.sigmoid(self.final_conv(x))
        return x

In [12]:
BATCH_SIZE = 2
train_dataloader = DataLoader(CustomDataset(22, 2694), batch_size=BATCH_SIZE, shuffle=True, num_workers=8)
val_dataloader = DataLoader(CustomDataset(5410, 364), batch_size=BATCH_SIZE, shuffle=False, num_workers=8)
test_dataloader = DataLoader(CustomDataset(6141, 912), batch_size=BATCH_SIZE, shuffle=False, num_workers=8)

In [13]:
train_steps = len(train_dataloader.dataset) // BATCH_SIZE
batch_mask = mask.repeat(BATCH_SIZE, 1, 1, 1)

In [14]:
eps = 1e-8
mask_sum = torch.sum(mask)
class WeightedBCELoss(nn.Module):
    def __init__(self, alpha=0.95):
        super().__init__()
        self.alpha = alpha
 
    def forward(self, output, target):
        output = torch.clamp(output, eps, 1. - eps)
        pt = (target * output) + ((1 - target) * (1 - output))
        BCE_loss = -torch.log(pt)
        BCE_loss = BCE_loss * mask
        alpha_t = (target * self.alpha) + ((1 - target) * (1 - self.alpha))
        final_loss = alpha_t * BCE_loss
        return torch.sum(final_loss) / mask_sum

eps = 1e-8
mask_sum = torch.sum(mask)
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.99, gamma=2):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
 
    def forward(self, output, target):
        output = torch.clamp(output, eps, 1. - eps)
        pt = (target * output) + ((1 - target) * (1 - output))
        BCE_loss = -torch.log(pt)
        pt = pt * mask
        BCE_loss = BCE_loss * mask
        alpha_t = (target * self.alpha) + ((1 - target) * (1 - self.alpha))
        F_loss = alpha_t * ((1 - pt) ** self.gamma) * BCE_loss
        return torch.sum(F_loss) / mask_sum



model = Net().to(device)
loss_fn = WeightedBCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

print(model)

Net(
  (convs): ModuleList(
    (0): Conv2d(508, 512, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): Conv2d(512, 256, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (2): Conv2d(256, 128, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (3): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (4): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (5): Conv2d(32, 16, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (6): Conv2d(16, 8, kernel_size=(3, 3), stride=(1, 1), padding=same)
  )
  (bns): ModuleList(
    (0): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_ru

In [15]:
def get_metrics(dataloader, threshold=0.5):
    model.eval()

    correct_num = 0
    correct_denom = 0
    recall_num = 0
    recall_denom = 0
    precision_num = 0
    precision_denom = 0
    with torch.no_grad():
        for inputs, labels, _ in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            y_pred = model(inputs)
            pred = y_pred > threshold

            pred = pred[batch_mask]
            labels = labels[batch_mask]
            
            correct_num += (pred == labels).float().sum().item()
            correct_denom += labels.numel()
            recall_num += (pred[labels > 0] == labels[labels > 0]).float().sum().item()
            recall_denom += labels[labels > 0].numel()
            precision_num += (pred[pred > 0] == labels[pred > 0]).float().sum().item()
            precision_denom += labels[pred > 0].numel()
        
        correct = 0
        if correct_denom > 0:
            correct = correct_num / correct_denom
        recall = 0
        if recall_denom > 0:
            recall = recall_num / recall_denom
        precision = 0
        if precision_denom > 0:
            precision = precision_num / precision_denom
        
        torch.cuda.empty_cache()
        
        return correct, recall, precision

In [21]:
log_dir = base_folder + "logs/retrain" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
writer = SummaryWriter(log_dir=log_dir)
print(log_dir)

2024-08-11 02:38:18.012338: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-11 02:38:18.519209: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


/scratch/w28/ad0033/FD/logs/retrain20240811-023817


In [ ]:
best_model_value = 0

n_epochs = 200
nan_found = False
for epoch in range(n_epochs):
    model.train()

    total_train_loss = 0
    train_correct = 0
    train_recall = 0
    train_precision = 0
    for inputs, labels, _ in tqdm(train_dataloader):
        inputs, labels = inputs.to(device), labels.to(device)
        
        y_pred = model(inputs)
        loss = loss_fn(y_pred, labels)

        if torch.isnan(loss):
            print(f"NaN detected in loss!")
            torch.save(model.state_dict(), curr_folder + "model_nan.pt")
            nan_found = True
            break
        
        optimizer.zero_grad()
        loss.backward()

        # Check for NaNs in gradients
        for name, param in model.named_parameters():
            if param.grad is not None:
                if torch.isnan(param.grad).any():
                    print(f"NaN detected in gradients of {name}")
                    nan_found = True
        if nan_found == True:
            break
    
        optimizer.step()

        # Check for NaNs in model weights
        for name, param in model.named_parameters():
            if torch.isnan(param).any():
                print(f"NaN detected in weights of {name}")
                nan_found = True
        if nan_found == True:
            break

        total_train_loss += loss

        # pred = torch.argmax(y_pred, dim=1)
        pred = y_pred > 0.5

        pred = pred[batch_mask]
        labels = labels[batch_mask]
        
        train_correct += ((pred == labels).float().sum().item() / labels.numel())
        if labels[labels > 0].numel() > 0:
            train_recall += ((pred[labels > 0] == labels[labels > 0]).float().sum().item() / labels[labels > 0].numel())
        if labels[pred > 0].numel() > 0:
            train_precision += ((pred[pred > 0] == labels[pred > 0]).float().sum().item() / labels[pred > 0].numel())
    
    if nan_found == True:
        break

    avg_train_loss = total_train_loss / train_steps
    train_correct = train_correct / train_steps
    train_recall = train_recall / train_steps
    train_precision = train_precision / train_steps
    
    print("Epoch: %d, Loss: %.4f" % (epoch+1, avg_train_loss))
    print("[Training Metrics]   Accuracy: %.4f, Recall: %.4f, Precision: %.4f" % (100*train_correct, 100*train_recall, 100*train_precision))
    
    with torch.no_grad():
        val_correct, val_recall, val_precision = get_metrics(val_dataloader)

    print("[Validation Metrics] Accuracy: %.4f, Recall: %.4f, Precision: %.4f" % (100*val_correct, 100*val_recall, 100*val_precision))
    
    writer.add_scalar('Loss/train', avg_train_loss.item(), epoch)
    writer.add_scalar('Accuracy/train', 100*train_correct, epoch)
    writer.add_scalar('Recall/train', 100*train_recall, epoch)
    writer.add_scalar('Precision/train', 100*train_precision, epoch)
    writer.add_scalar('Accuracy/validation', 100*val_correct, epoch)
    writer.add_scalar('Recall/validation', 100*val_recall, epoch)
    writer.add_scalar('Precision/validation', 100*val_precision, epoch)

    model_value = 0
    if (val_recall + val_precision) > 1e-4:
        model_value = 2 * (val_recall * val_precision) / (val_recall + val_precision) # F1 Score
    print("Model Value:", model_value)
    if model_value > best_model_value:
        print("Saving Model!")
        torch.save(model.state_dict(), curr_folder + "model.pt")
        best_model_value = model_value

100%|██████████| 1347/1347 [21:03<00:00,  1.07it/s]


Epoch: 1, Loss: 0.0651
[Training Metrics]   Accuracy: 84.8755, Recall: 54.4794, Precision: 7.4213
[Validation Metrics] Accuracy: 78.5508, Recall: 83.0984, Precision: 7.0587
Model Value: 0.13012175797639108
Saving Model!


100%|██████████| 1347/1347 [20:50<00:00,  1.08it/s]


Epoch: 2, Loss: 0.0584
[Training Metrics]   Accuracy: 86.4994, Recall: 61.1278, Precision: 9.0660
[Validation Metrics] Accuracy: 87.6945, Recall: 76.5679, Precision: 11.0880
Model Value: 0.19370928219936348
Saving Model!


100%|██████████| 1347/1347 [21:06<00:00,  1.06it/s]


...

100%|██████████| 1347/1347 [20:59<00:00,  1.07it/s]


Epoch: 95, Loss: 0.0249
[Training Metrics]   Accuracy: 93.9837, Recall: 80.2938, Precision: 20.2717
[Validation Metrics] Accuracy: 93.6402, Recall: 75.3540, Precision: 19.8228
Model Value: 0.3138845513842383


100%|██████████| 1347/1347 [21:00<00:00,  1.07it/s]


Epoch: 96, Loss: 0.0250
[Training Metrics]   Accuracy: 94.0503, Recall: 80.8208, Precision: 20.7101
[Validation Metrics] Accuracy: 92.7881, Recall: 70.8503, Precision: 17.0614
Model Value: 0.2750038669060681


 79%|███████▉  | 1065/1347 [16:33<04:22,  1.07it/s]

In [ ]:
print("Best Model Value:", best_model_value)

In [16]:
checkpoint = torch.load(curr_folder + "model.pt")
model.load_state_dict(checkpoint)
model.eval()

Net(
  (convs): ModuleList(
    (0): Conv2d(508, 512, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): Conv2d(512, 256, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (2): Conv2d(256, 128, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (3): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (4): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (5): Conv2d(32, 16, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (6): Conv2d(16, 8, kernel_size=(3, 3), stride=(1, 1), padding=same)
  )
  (bns): ModuleList(
    (0): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_ru

In [17]:
def f1_score(recall, precision):
    return 2 * (recall * precision) / (recall + precision + 1e-06)

In [17]:
val_correct, val_recall, val_precision = get_metrics(val_dataloader)

print("[Validation Metrics] Accuracy: %.4f, Recall: %.4f, Precision: %.4f" % (100*val_correct, 100*val_recall, 100*val_precision))

print("Model Value:", f1_score(val_recall, val_precision))

[Validation Metrics] Accuracy: 96.2766, Recall: 63.6641, Precision: 28.9121
Model Value: 0.39765404295060053


In [18]:
test_correct, test_recall, test_precision = get_metrics(test_dataloader)

print("[Test Metrics] Accuracy: %.4f, Recall: %.4f, Precision: %.4f" % (100*test_correct, 100*test_recall, 100*test_precision))

[Test Metrics] Accuracy: 94.5597, Recall: 65.0572, Precision: 29.2722


In [22]:
train_correct, train_recall, train_precision = get_metrics(train_dataloader)

print("[Train Metrics] Accuracy: %.4f, Recall: %.4f, Precision: %.4f" % (100*train_correct, 100*train_recall, 100*train_precision))

[Train Metrics] Accuracy: 96.2932, Recall: 82.6073, Precision: 33.7189
